<a href="https://colab.research.google.com/github/anky19698/Cricket_Analytics/blob/main/Batting_Analysis_IPL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Problem Statement:
# How good is player X against MI in Wankhede when they’re playing 3 spinners?

In [246]:
import pandas as pd
import numpy as np

# **Functions**

In [247]:
def balls_per_dismissal(balls, dismissals):
    if dismissals > 0:
        return balls/dismissals
    else:
        return balls

def balls_per_boundary(balls, boundaries):
    if boundaries > 0:
        return balls/boundaries
    else:
        return balls

In [248]:
def custom_filter(data, venue, phase, opposition):




  data = data[data.venue == venue]
  data = data[data.phase == phase]
  data = data[data.bowling_team == opposition]


  data['is_dot'] = data['batsman_runs'].apply(lambda x: 1 if x == 0 else 0)
  data['is_one'] = data['batsman_runs'].apply(lambda x: 1 if x == 1 else 0)
  data['is_two'] = data['batsman_runs'].apply(lambda x: 1 if x == 2 else 0)
  data['is_three'] = data['batsman_runs'].apply(lambda x: 1 if x == 3 else 0)
  data['is_four'] = data['batsman_runs'].apply(lambda x: 1 if x == 4 else 0)
  data['is_six'] = data['batsman_runs'].apply(lambda x: 1 if x == 6 else 0)


  dots = pd.DataFrame(data.groupby(['batsman'])['is_dot'].sum()).reset_index().rename(columns = {'is_dot':'dots'})
  ones = pd.DataFrame(data.groupby(['batsman'])['is_one'].sum()).reset_index().rename(columns = {'is_one':'ones'})
  twos = pd.DataFrame(data.groupby(['batsman'])['is_two'].sum()).reset_index().rename(columns = {'is_two':'twos'})
  threes = pd.DataFrame(data.groupby(['batsman'])['is_three'].sum()).reset_index().rename(columns = {'is_three':'threes'})

  runs = pd.DataFrame(data.groupby(['batsman'])['batsman_runs'].sum()).reset_index().rename(columns = {'batsman_runs': 'runs'})
  balls = pd.DataFrame(data.groupby(['batsman'])['match_id'].count()).reset_index().rename(columns = {'match_id': 'balls'})
  innings = pd.DataFrame(data.groupby(['batsman'])['match_id'].apply(lambda x: len(list(np.unique(x)))).reset_index().rename(columns = {'match_id': 'innings'}))
  dismissals = pd.DataFrame(data.groupby(['batsman'])['player_dismissed'].count()).reset_index().rename(columns = {'player_dismissed':'dismissals'})
  fours = data.groupby(['batsman'])['is_four'].sum().reset_index().rename(columns = {'is_four': 'fours'})
  sixes = data.groupby(['batsman'])['is_six'].sum().reset_index().rename(columns = {'is_six': 'sixes'})

  batting = pd.merge(innings, runs, on='batsman').merge(balls, on='batsman').merge(dismissals, on='batsman').merge(fours, on='batsman').merge(sixes, on='batsman').merge(dots, on='batsman').merge(ones, on='batsman').merge(twos, on='batsman').merge(threes, on='batsman')

  # batting['Strike Rate'] = batting.apply(lambda x: (100*x['runs']/x['balls']), axis=1)
  # batting['Runs Per Innings'] = batting.apply(lambda x: (x['runs']/x['innings']), axis=1)

  #StrikeRate
  batting['SR'] = batting.apply(lambda x: 100*(x['runs']/x['balls']), axis = 1)

  #runs per innings
  batting['RPI'] = batting.apply(lambda x: x['runs']/x['innings'], axis = 1)

  #balls per dismissals
  batting['BPD'] = batting.apply(lambda x: balls_per_dismissal(x['balls'], x['dismissals']), axis = 1)

  #balls per boundary
  batting['BPB'] = batting.apply(lambda x: balls_per_boundary(x['balls'], (x['fours'] + x['sixes'])), axis = 1)

  batting['dot_percentage'] = batting.apply(lambda x: x['dots']/x['balls'], axis = 1)

  return batting


In [249]:
def get_phase(over):
  if over <= 6:
    return 'Powerplay'
  elif over > 6 and over < 16:
    return 'Middle'
  else:
    return 'Death'



## Importing IPL Data 2008-2023

In [250]:
matches = pd.read_csv('/content/matches.csv')

In [251]:
deliveries = pd.read_csv('/content/deliveries.csv')

In [252]:
df = deliveries.copy()

df = df.rename(columns = {'matchId':'match_id'})

df.head()

,match_id,inning,over_ball,over,ball,batting_team,bowling_team,batsman,non_striker,bowler,batsman_runs,extras,isWide,isNoBall,Byes,LegByes,Penalty,dismissal_kind,player_dismissed
0,1082591,1,0.1,0,1,Sunrisers Hyderabad,Royal Challengers Bangalore,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,-,-
1,1082591,1,0.2,0,2,Sunrisers Hyderabad,Royal Challengers Bangalore,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,-,-
2,1082591,1,0.3,0,3,Sunrisers Hyderabad,Royal Challengers Bangalore,DA Warner,S Dhawan,TS Mills,4,0,0,0,0,0,0,-,-
3,1082591,1,0.4,0,4,Sunrisers Hyderabad,Royal Challengers Bangalore,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,-,-
4,1082591,1,0.5,0,5,Sunrisers Hyderabad,Royal Challengers Bangalore,DA Warner,S Dhawan,TS Mills,0,2,2,0,0,0,0,-,-


In [253]:
mdf = matches.copy()

mdf = mdf.rename(columns = {'matchId':'match_id'})

In [254]:
mdf['match_id'].count()

1024

In [255]:
final = pd.merge(df, mdf, on='match_id', how='left')

In [256]:
final.columns


Index(['match_id', 'inning', 'over_ball', 'over', 'ball', 'batting_team',
       'bowling_team', 'batsman', 'non_striker', 'bowler', 'batsman_runs',
       'extras', 'isWide', 'isNoBall', 'Byes', 'LegByes', 'Penalty',
       'dismissal_kind', 'player_dismissed', 'outcome', 'event', 'date',
       'date1', 'date2', 'gender', 'reserve_umpire', 'umpire1', 'umpire2',
       'match_referee', 'tv_umpire', 'season', 'venue', 'city',
       'toss_decision', 'toss_winner', 'team1', 'team2', 'winner_runs',
       'balls_per_over', 'player_of_match', 'neutralvenue', 'method', 'winner',
       'match_number', 'eliminator', 'winner_wickets'],
      dtype='object')

In [257]:
final['phase'] = final['over'].apply(lambda x: get_phase(x))

## Adding Year

In [258]:
# Convert 'date_column' to datetime
final['date_played'] = pd.to_datetime(final['date'])

# Extract the year
final['year'] = final['date_played'].dt.year

# final['year'].unique()

# final = final[final['year'] > 2021]

In [259]:
final['player_dismissed'].replace('-', pd.NA, inplace=True)

In [260]:
final.groupby(['batsman'])['player_dismissed'].count()

batsman
A Ashish Reddy     15
A Badoni           18
A Chandila          1
A Chopra            5
A Choudhary         2
                 ... 
Yashpal Singh       4
Younis Khan         1
Yudhvir Singh       2
Yuvraj Singh      109
Z Khan             15
Name: player_dismissed, Length: 637, dtype: int64

In [261]:
final_df = custom_filter(final, 'Wankhede Stadium', 'Middle', 'Mumbai Indians')

# Assignment – Using the latest data, perform the same analysis with opposition as CSK, venue as Chepauk.

In [276]:
final_df = custom_filter(final, 'MA Chidambaram Stadium, Chepauk', 'Middle', 'Chennai Super Kings')

In [277]:
final_df.head()

,batsman,innings,runs,balls,dismissals,fours,sixes,dots,ones,twos,threes,SR,RPI,BPD,BPB,dot_percentage
0,A Ashish Reddy,1,1,2,0,0,0,1,1,0,0,50.000000,1.0,2.0,2.0,0.500000
1,A Mishra,1,6,11,1,0,0,6,4,1,0,54.545455,6.0,11.0,11.0,0.545455
2,A Mithun,1,11,8,1,2,0,3,3,0,0,137.500000,11.0,8.0,4.0,0.375000
3,A Symonds,1,35,30,0,2,2,12,13,1,0,116.666667,35.0,30.0,7.5,0.400000
4,AB Agarkar,1,7,4,0,1,0,1,1,1,0,175.000000,7.0,4.0,4.0,0.250000


In [278]:
final_df.head()

,batsman,innings,runs,balls,dismissals,fours,sixes,dots,ones,twos,threes,SR,RPI,BPD,BPB,dot_percentage
0,A Ashish Reddy,1,1,2,0,0,0,1,1,0,0,50.000000,1.0,2.0,2.0,0.500000
1,A Mishra,1,6,11,1,0,0,6,4,1,0,54.545455,6.0,11.0,11.0,0.545455
2,A Mithun,1,11,8,1,2,0,3,3,0,0,137.500000,11.0,8.0,4.0,0.375000
3,A Symonds,1,35,30,0,2,2,12,13,1,0,116.666667,35.0,30.0,7.5,0.400000
4,AB Agarkar,1,7,4,0,1,0,1,1,1,0,175.000000,7.0,4.0,4.0,0.250000


# Topsis Method

In [279]:
wt_sr, wt_rpi, wt_bpd, wt_dot_percentage = 0.13, 0.27, 0.16, 0.45

In [280]:
df = final_df.copy()

## Applying **Filters**

In [281]:
# Batsmen Who Played Atleast 3 Innings and 30 Balls

df = df[(df.innings >= 3) & (df.balls >= 30 ) ]


## Topsis Calculation

In [282]:
#step1: square of all values
df['calc_SR'] = df['SR'].apply(lambda x: x*x)
df['calc_RPI'] = df['RPI'].apply(lambda x: x*x)
df['calc_BPD'] = df['BPD'].apply(lambda x: x*x)
df['calc_dot_percentage'] = df['dot_percentage'].apply(lambda x: x*x)

#step2: square root of all values
sq_sr, sq_rpi, sq_bpd, sq_dot_percentage = np.sqrt(df[['calc_SR','calc_RPI', 'calc_BPD', 'calc_dot_percentage']].sum(axis = 0))

df['calc_SR'] = df['calc_SR'].apply(lambda x: x/sq_sr)
df['calc_RPI'] = df['calc_RPI'].apply(lambda x: x/sq_rpi)
df['calc_BPD'] = df['calc_BPD'].apply(lambda x: x/sq_bpd)
df['calc_dot_percentage'] = df['calc_dot_percentage'].apply(lambda x: x/sq_dot_percentage)

df['calc_SR'] = df['calc_SR'].apply(lambda x: x*wt_sr)
df['calc_RPI'] = df['calc_RPI'].apply(lambda x: x*wt_rpi)
df['calc_BPD'] = df['calc_BPD'].apply(lambda x: x*wt_bpd)
df['calc_dot_percentage'] = df['calc_dot_percentage'].apply(lambda x: x*wt_dot_percentage)

best_sr, worst_sr = max(df['calc_SR']), min(df['calc_SR'])
best_rpi, worst_rpi = max(df['calc_RPI']), min(df['calc_RPI'])
best_bpd, worst_bpd = max(df['calc_BPD']), min(df['calc_BPD'])
best_dot_percentage, worst_dot_percentage = min(df['calc_dot_percentage']), max(df['calc_dot_percentage'])

In [283]:
df['dev_best_SR'] = df['calc_SR'].apply(lambda x: (x-best_sr)*(x-best_sr))
df['dev_best_RPI'] = df['calc_RPI'].apply(lambda x: (x-best_rpi)*(x-best_rpi))
df['dev_best_BPD'] = df['calc_BPD'].apply(lambda x: (x-best_bpd)*(x-best_bpd))
df['dev_best_dot_percentage'] = df['calc_dot_percentage'].apply(lambda x: (x-best_dot_percentage)*(x-best_dot_percentage))

df['dev_best_sqrt'] = df.apply(lambda x: x['dev_best_SR'] + x['dev_best_RPI'] + x['dev_best_BPD'] + x['dev_best_dot_percentage'], axis = 1)

df['dev_worst_SR'] = df['calc_SR'].apply(lambda x: (x-worst_sr)*(x-worst_sr))
df['dev_worst_RPI'] = df['calc_RPI'].apply(lambda x: (x-worst_rpi)*(x-worst_rpi))
df['dev_worst_BPD'] = df['calc_BPD'].apply(lambda x: (x-worst_bpd)*(x-worst_bpd))
df['dev_worst_dot_percentage'] = df['calc_dot_percentage'].apply(lambda x: (x-worst_dot_percentage)*(x-worst_dot_percentage))

df['dev_worst_sqrt'] = df.apply(lambda x: x['dev_worst_SR'] + x['dev_worst_RPI'] + x['dev_worst_BPD'] + x['dev_worst_dot_percentage'], axis = 1)


In [284]:
df['score'] = df.apply(lambda x: x['dev_worst_sqrt']/(x['dev_worst_sqrt'] + x['dev_best_sqrt']), axis = 1)

## End Result: Top players for our required role at Chepauk Against CSK

In [285]:
chennai_ps = pd.DataFrame(df[['batsman', 'innings', 'runs', 'balls', 'dismissals', 'score']].sort_values(['score'], ascending=False))

chennai_ps

,batsman,innings,runs,balls,dismissals,score
102,SE Marsh,3,93,50,1,0.896916
5,AB de Villiers,3,99,67,0,0.819686
40,G Gambhir,3,80,72,1,0.521570
81,NV Ojha,3,67,41,2,0.500277
107,SR Watson,4,105,66,3,0.494111
74,MS Bisla,3,82,57,2,0.379636
118,Y Venugopal Rao,3,65,49,2,0.212676
113,V Kohli,7,172,137,5,0.189949
27,DA Miller,3,47,36,2,0.157593
72,MK Tiwary,3,49,45,1,0.129532


## End Result: Top players for our required role at Wankhede Against MI

In [272]:
mumbai_ps = pd.DataFrame(df[['batsman', 'innings', 'runs', 'balls', 'dismissals', 'score']].sort_values(['score'], ascending=False))

mumbai_ps

,batsman,innings,runs,balls,dismissals,score
136,SV Samson,4,134,87,1,0.826816
49,GJ Maxwell,4,93,46,4,0.549684
71,KL Rahul,3,89,79,1,0.479964
127,SE Marsh,4,147,99,2,0.438758
4,AB de Villiers,6,174,99,4,0.432944
87,MK Tiwary,3,86,71,1,0.410723
37,DJ Bravo,6,74,70,1,0.254479
145,WP Saha,3,69,49,1,0.243351
26,CH Gayle,3,87,60,2,0.223017
19,BB McCullum,3,76,51,2,0.210053
